# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm_model = "gpt-4.1-nano"

In [13]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [14]:
conversation.invoke(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


{'input': 'Hi, my name is Andrew',
 'history': '',
 'response': "Hello, Andrew! It's great to meet you. How can I assist you today?"}

In [15]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


"1 plus 1 equals 2. It's a simple addition problem, and the answer is straightforward. If you'd like, I can provide more details about addition or help with other math questions!"

In [16]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 plus 1 equals 2. It's a simple addition problem, and the answer is straightforward. If you'd like, I can provide more details about addition or help with other math questions!
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

In [17]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 plus 1 equals 2. It's a simple addition problem, and the answer is straightforward. If you'd like, I can provide more details about addition or help with other math questions!
Human: What is my name?
AI: Your name is Andrew.


In [18]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello, Andrew! It's great to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1 plus 1 equals 2. It's a simple addition problem, and the answer is straightforward. If you'd like, I can provide more details about addition or help with other math questions!\nHuman: What is my name?\nAI: Your name is Andrew."}

In [19]:
memory = ConversationBufferMemory()

In [20]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [21]:
print(memory.buffer)

Human: Hi
AI: What's up


In [22]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [23]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [24]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [25]:
from langchain.memory import ConversationBufferWindowMemory

In [26]:
memory = ConversationBufferWindowMemory(k=1)               

C:\Users\Fabien CHERER\AppData\Local\Temp\ipykernel_2768\802562642.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


In [27]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [28]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [29]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [30]:
conversation.predict(input="Hi, my name is Andrew")

"Hello, Andrew! It's great to meet you. How can I assist you today?"

In [31]:
conversation.predict(input="What is 1+1?")

"1 plus 1 equals 2. It's a simple addition problem, and the sum of these two numbers is 2. If you'd like, I can help you with more math questions or anything else you're curious about!"

In [32]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't know your name unless you tell me. If you'd like to share it, I'd be happy to remember it for our conversation!"

## ConversationTokenBufferMemory

In [ ]:
#!pip install tiktoken

In [36]:
from langchain.memory import ConversationTokenBufferMemory
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

In [37]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [38]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [39]:
from langchain.memory import ConversationSummaryBufferMemory

In [40]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

C:\Users\Fabien CHERER\AppData\Local\Temp\ipykernel_2768\1387621936.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)


In [41]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI, and they exchange casual pleasantries. The human mentions they are just hanging out and asks what is on the schedule for the day.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [42]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [43]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI, and they exchange casual pleasantries. The human mentions they are just hanging out and asks what is on the schedule for the day.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


"A great demo to show would be a conversational AI application that utilizes a large language model (LLM) to answer questions in real-time. You could set up a scenario where the AI assists with common customer inquiries, showcasing its ability to understand context and provide relevant information. \n\nAnother option could be a text generation demo where you input a prompt, and the LLM generates a creative story or a professional email. This would highlight the model's versatility and creativity. \n\nIf you want to focus on practical applications, consider demonstrating how the LLM can summarize long documents or extract key information from a dataset. This could be particularly relevant for your customer, as it shows how AI can streamline workflows and enhance productivity. \n\nMake sure to prepare a few different scenarios to illustrate the model's capabilities, and perhaps even include a live interaction where the customer can ask questions to the AI themselves!"

In [44]:
memory.load_memory_variables({})

{'history': "System: The human greets the AI, and they exchange casual pleasantries. The human mentions they are just hanging out and asks what is on the schedule for the day. The AI outlines the day's agenda, including a meeting with the product team at 8am, time to work on the LangChain project, lunch with a customer at noon, and the need to bring a laptop for a demo. The human asks for a good demo to show, and the AI suggests a conversational AI application using a large language model (LLM) to answer customer inquiries, a text generation demo, or a practical application like summarizing documents. The AI emphasizes preparing various scenarios and including a live interaction for the customer to engage with the AI."}